In [1]:
import ast
import syft as sy
from syft.core.node.common.action.function_or_constructor_action import (
    RunFunctionOrConstructorAction,
)
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.node.common.action.run_class_method_action import RunClassMethodAction
from syft.core.pointer.pointer import Pointer

In [3]:
# use new syckle secure custom code delivery and remote execution

In [4]:
domain = sy.Domain(name="Bob")
root_client = domain.get_root_client()

In [5]:
# # first we must create the remote code

# our_code = """
# class Test:
#     def hello(self) -> str:
#         return "syckle hello"
# """

# send_custom_class_code(root_client, our_code,"Test")

In [6]:
# # now we just use these custom functions to execute without knowledge of the AST
# test_ptr = init_class(root_client, "syft.sandbox.mylib.Test")
# hello_ptr = run_on(test_ptr, "hello")

In [7]:
# hello_ptr.get(delete_obj=False)

In [8]:
# root_client.store.pandas

In [9]:
# import sys
# sys.modules["syft"].sandbox
# sy.sandbox.mylib.Test

In [10]:
import inspect, sys

def new_getfile(object, _old_getfile=inspect.getfile):
    if not inspect.isclass(object):
        return _old_getfile(object)
    
    # Lookup by parent module (as in current inspect)
    if hasattr(object, '__module__'):
        object_ = sys.modules.get(object.__module__)
        if hasattr(object_, '__file__'):
            return object_.__file__
    
    # If parent module is __main__, lookup by methods (NEW)
    for name, member in inspect.getmembers(object):
        if inspect.isfunction(member) and object.__qualname__ + '.' + member.__name__ == member.__qualname__:
            return inspect.getfile(member)
    else:
        raise TypeError('Source for {!r} not found'.format(object))
inspect.getfile = new_getfile

import inspect
from IPython.core.magics.code import extract_symbols

def get_code(obj):

    try:
        return inspect.getsource(obj)
    except Exception as e:
        # in case we're looking for a class that was generated within a
        # Jupyter notebook we use the following
        cell_code = "".join(inspect.linecache.getlines(new_getfile(obj)))
        class_code = extract_symbols(cell_code, obj.__name__)[0][0]
        return class_code

In [11]:
code_str = """
## USER CODE

{custom_code}

## Sandboxing Code
import syft
module_type = type(syft)

if "sandbox" not in syft.__dict__:
    syft.__dict__["sandbox"] = module_type(name="sandbox")

parent = syft.__dict__["sandbox"]

namespace = "mylib"

import_path = f"{namespace}.{class_name}"
module_parts = import_path.split(".")

klass = module_parts.pop()
{class_name}.__module__ = f"syft.sandbox.{namespace}"
{class_name}.__name__ = klass

for n in module_parts:
    if n not in parent.__dict__:
        parent.__dict__[n] = module_type(name=n)
    parent = parent.__dict__[n]

parent.__dict__[{class_name}.__name__] = {class_name}

print("done", syft.sandbox.mylib)
"""

def send_custom_class_code(client, code, class_name):
    code = code_str.replace("{custom_code}", code)
    code = code.replace("{class_name}", class_name)
    tree = ast.parse(code)
    client.secure_exec(tree)

In [12]:
class ClassCode():
    
    def __init__(self, code, class_name) -> None:
        self.code = code
        self.class_name = class_name
        
    def send(self, destination_client):
        return send_custom_class_code(destination_client, self.code, self.class_name)
        
    def __repr__(self):
        return self.code

In [13]:
def sendable(func_or_class):
    return ClassCode(get_code(func_or_class).replace("@sendable\n",""), func_or_class.__name__)

In [35]:
@sendable
def myfunc():
    return 3

@sendable
class MyClass(object):
    test:int = 2
    
    def __init__(self):
        self.L = 5
    
    def test(self, x:str) -> str:
        return True
    
    @classmethod
    def forward(cls, x):
        return x
    
    def hello(self) -> str:
        return "syckle helasdfasdfa"    
    
@sendable
class Test2:
    def hello(self) -> str:
        return "syckle helasdfasdfa"

In [37]:
Test2.send(root_client)

Remote Secure Exec
Compiling...
done <module 'mylib'>
... compiled!
Accepting, code executed successfully!


In [38]:
def init_class(client, path: str):
    action = RunFunctionOrConstructorAction(
        path="syft.sandbox.mylib.Test",
        args=tuple(),
        kwargs={},
        id_at_location=UID(),
        address=client.address,
        msg_id=UID(),
    )

    client.send_immediate_msg_without_reply(msg=action)

    module_parts = path.split(".")
    klass_name = module_parts.pop()
    # syft.proxy.torch.TensorPointer
    fake_pointer = Pointer(id_at_location=action.id_at_location, client=client)
    fake_pointer.path_and_name=action.path
    fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
    fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
    fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
    return fake_pointer

def run_on(pointer, method: str, args = [], kwargs = {}):
    action = RunClassMethodAction(
        path=f"{pointer.path_and_name}.{method}",
        _self=pointer,
        args=args,
        kwargs=kwargs,
        id_at_location=UID(),
        address=pointer.client.address,
        msg_id=UID(),
    )
    pointer.client.send_immediate_msg_without_reply(msg=action)

    fake_pointer = Pointer(id_at_location=action.id_at_location, client=pointer.client)
    fake_pointer.path_and_name = action.path
#     fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
#     fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
#     fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
    return fake_pointer

In [39]:
test_ptr = init_class(root_client, "syft.sandbox.mylib.Test2")
hello_ptr = run_on(test_ptr, "hello")

[2021-06-16T21:28:18.755442+0100][CRITICAL][logger]][26535] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-06-16T21:28:18.755994+0100][CRITICAL][logger]][26535] 'Class' object has no attribute 'Test2Pointer'
[2021-06-16T21:28:18.756467+0100][CRITICAL][logger]][26535] '__getattr__ failed, Test2Pointer is not present on the object, nor the AST attributes!'


KeyError: '__getattr__ failed, Test2Pointer is not present on the object, nor the AST attributes!'

In [27]:
# run_on(test_ptr, "__init__")

In [22]:
hello_ptr.get()

'syckle helasdfasdfa'